In [1]:
import ollama
import json
from openai import OpenAI
import gradio as gr

In [2]:
!ollama list

NAME              ID              SIZE      MODIFIED     
gpt-oss:20b       aa4295ac10c3    13 GB     2 days ago      
mistral:latest    6577803aa9a0    4.4 GB    4 weeks ago     
llama3.1:8b       46e0c10c039e    4.9 GB    2 months ago    


In [3]:
MODEL = "llama3.1:8b"

openai = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [7]:
ticket_prices = {"london": "r$5600", "paris": "R$4350", "tokyo": "R$3670", "berlin": "R$4635"}

def get_ticket_price(destination_city):
    print(f"Ferramenta get_ticket_price called on {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [8]:
get_ticket_price("TOKYO")

Ferramenta get_ticket_price called on TOKYO


'R$3670'

In [10]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for examine",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [11]:
tools = [{"type": "function", "function": price_function}]

In [15]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [14]:
def handle_tool_call(message):
    tool_call=message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat).launch()

c:\Users\asus\anaconda3\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Ferramenta get_ticket_price called on Tokyo
Ferramenta get_ticket_price called on Tokyo
Ferramenta get_ticket_price called on Tokyo
Ferramenta get_ticket_price called on Berlin
Ferramenta get_ticket_price called on Berlin
Ferramenta get_ticket_price called on Brasil
